In [1]:
import requests
from requests import Session
import pandas as pd

# Public API

In [2]:
class MarketData:
    # https://docs.kucoin.com/ - API Documentaion

    def __init__(self):
        self.apiurl = "https://api.kucoin.com"  # base url for kucoin
        self.session = Session()

    # returns all tickers
    def getAllTickers(self):
        url = self.apiurl + "/api/v1/market/allTickers"
        r = self.session.get(url)
        data = r.json()["data"]["ticker"]
        df = pd.DataFrame.from_dict(data, orient="columns")
        return df.loc[
            :,
            [
                "symbolName",
                "buy",
                "sell",
                "changeRate",
                "changePrice",
                "high",
                "low",
                "vol",
                "last",
                "averagePrice",
            ],
        ]

    # returns a single trading pair ex. 'BTC-USDT'
    def getTicker(self, symbol):
        url = self.apiurl + "/api/v1/market/orderbook/level1"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()["data"]
        # dictionary to dataframe
        df = pd.DataFrame.from_dict(data, orient="index", columns=[symbol])
        return df
    
    def get24hrStats(self, symbol):
        url = self.apiurl + "/api/v1/market/stats"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()["data"]
        # dictionary to dataframe
        df = pd.DataFrame.from_dict(data, orient="index", columns=[symbol])
        return df

    # returns the order book for a trading pair
    def getPartOrderBook(self, symbol, amount):  # does not required API KEY
        # amount can be 20 pieces of data or 100 pieces of data
        url = self.apiurl + "/api/v1/market/orderbook/level2_" + amount
        parameters = {"symbol": symbol, "amount": amount}
        r = self.session.get(url, params=parameters)
        bids = r.json()["data"]["bids"]
        asks = r.json()["data"]["asks"]
        dfBids = pd.DataFrame.from_dict(bids, orient="columns")
        dfAsks = pd.DataFrame.from_dict(asks, orient="columns")
        dfAsks.columns = ["", "Asks"]
        dfBids.columns = ["", "Bids"]
        return dfBids, dfAsks

    # returns the fiat price of a symbol
    def getFiatPrice(self, symbol):
        url = self.apiurl + "/api/v1/prices"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()["data"][symbol]
        return data
    
    def getAllFutureContracts(self):
        url = 'https://api-futures.kucoin.com' + '/api/v1/contracts/active'
        r = self.session.get(url)
        data = r.json()['data']
        return data

In [3]:
kucoinAPI = MarketData()

In [4]:
kucoinAPI.getAllTickers()

,symbolName,buy,sell,changeRate,changePrice,high,low,vol,last,averagePrice
0,NKN-USDT,0.114115,0.114563,0.0385,0.004232,0.115293,0.108424,283558.01724246,0.114053,0.10959163
1,LOOM-BTC,0.0000022,0.000002204,0.0286,0.000000061,0.000002204,0.000002103,137925.0136,0.000002187,0.00000211
2,JUP-ETH,0.000007952,0.000007973,-0.0527,-0.000000443,0.000008524,0.00000796,1378729.6603,0.00000796,0.0000082
3,CUSD-USDT,0.9966,0.997,0.0003,0.0003,0.9979,0.9936,72200.79241768,0.9963,0.9952568
4,LTC3L-USDT,0.003849,0.003872,0.0197,0.000075,0.004,0.003632,8840409.07618299,0.003869,0.00370552
...,...,...,...,...,...,...,...,...,...,...
1262,FRR-USDT,0.00387,0.0039,0.0427,0.00016,0.00415,0.00344,111500844.09543393,0.0039,0.00370325
1263,DMTR-USDT,0.02912,0.02926,-0.0247,-0.00074,0.03007,0.028,18261570.4493516,0.0292,0.02940013
1264,GMB-BTC,0.0000000122,0.0000000123,0.0166,0.0000000002,0.0000000124,0.0000000115,19307728.5169,0.0000000122,0.00000001
1265,TRIBE-USDT,0.1602,0.1609,0.0031,0.0005,0.1617,0.1588,60165.1213,0.1602,0.16013926


In [5]:
kucoinAPI.getTicker('ETH-USDT')

,ETH-USDT
time,1659755279241
sequence,1632889612212
price,1740.2
size,0.00287093
bestBid,1740.19
bestBidSize,3.3005619
bestAsk,1740.2
bestAskSize,0.95432797


In [6]:
kucoinAPI.getPartOrderBook('ETH-USDT', '20')

(                  Bids
 0   1739.58  0.0015873
 1   1739.56   2.061236
 2   1739.51  0.1861179
 3   1739.35  7.2575429
 4   1739.34    5.77014
 5   1739.31  1.5619999
 6    1739.3  1.5749999
 7   1739.27  5.8080145
 8   1739.26  2.8741579
 9   1739.24    7.97132
 10  1739.21  2.0028466
 11  1739.17  0.2670486
 12  1739.15    2.00118
 13  1739.11  0.2385786
 14  1739.06     7.5071
 15  1739.05        6.4
 16  1739.02  0.2850742
 17     1739  2.1042669
 18  1738.98  4.5271103
 19  1738.97  0.2243622,
                    Asks
 0   1739.59  10.5208806
 1   1739.72   0.0216284
 2   1739.74   5.9320283
 3   1739.75     6.96535
 4   1739.77      7.5071
 5   1739.78   0.0013163
 6   1739.84     1.19521
 7   1739.85   7.1714538
 8   1739.86     6.96535
 9   1739.96   0.0022306
 10  1739.97   0.6736144
 11  1739.98     6.96535
 12     1740   6.3216738
 13  1740.01   0.9668666
 14  1740.05    4.068335
 15  1740.06     5.97014
 16  1740.09   5.9691256
 17  1740.11     8.56535
 18  1740.12      0.

In [7]:
kucoinAPI.getFiatPrice('ICP')

'8.40083993'

In [8]:
kucoinAPI.get24hrStats('ICP-USDT')

,ICP-USDT
time,1659755286244
symbol,ICP-USDT
buy,8.4
sell,8.41
changeRate,0.0231
changePrice,0.19
high,8.42
low,7.91
vol,217331.69846611
volValue,1768713.4295801651
